# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
499,YUM
500,ZBH
501,ZBRA
502,ZION


In [6]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [7]:
symbol = 'AAPL'

baseUrl = f'https://sandbox.iexapis.com/stable'

quoteUrl = f'/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
apiUrl = baseUrl + quoteUrl

data = requests.get(apiUrl).json()
data

{'avgTotalVolume': 91520160,
 'calculationPrice': 'close',
 'change': 1.6,
 'changePercent': 0.01162,
 'close': 0,
 'closeSource': 'foaicfil',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': 'sCole',
 'highTime': 1681215954682,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 135.22,
 'iexCloseTime': 1690094164255,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 135.65,
 'iexOpenTime': 1732212347163,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1665936138055,
 'latestPrice': 132.14,
 'latestSource': 'Close',
 'latestTime': 'June 17, 2022',
 'latestUpdate': 1708963359377,
 'latestVolume': None,
 'low': 0,
 'lowSource': 'solCe',
 'lowTime': 1668925426800,
 'marketCap': 22223

In [8]:
data['latestPrice']

132.14

In [10]:
data['peRatio']

21.77

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [11]:
price = data['latestPrice']
peRatio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [14]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbolGroups = list(chunks(stocks['Ticker'], 100))
symbolStrings = []
for i in range(0, len(symbolGroups)):
    symbolStrings.append(','.join(symbolGroups[i]))
#     print(symbol_strings[i])

myColumns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [53]:
finalDataFrame = pd.DataFrame(columns = myColumns)

for symbolString in symbolStrings:
    batchApiUrlCall = baseUrl + f'/stock/market/batch?symbols={symbolString}&types=price,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchApiUrlCall).json()
    for symbol in symbolString.split(','):
        finalDataFrame = finalDataFrame.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = myColumns
            ),
            ignore_index = True
        )

finalDataFrame

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,115.85,27.27,N/A
1,AAL,13.34,-3.63,N/A
2,AAP,175.39,19.11,N/A
3,AAPL,132.74,21.65,N/A
4,ABBV,140.67,20.7,N/A
...,...,...,...,...
499,YUM,110.50,20.78,N/A
500,ZBH,103.30,103.71,N/A
501,ZBRA,289.61,19.13,N/A
502,ZION,52.53,5.5,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [46]:
finalDataFrame.sort_values(
    'Price-to-Earnings Ratio',
    ascending = True,
    inplace = True
)
finalDataFrame = finalDataFrame[finalDataFrame['Price-to-Earnings Ratio'] > 0]
finalDataFrame = finalDataFrame[:50]
finalDataFrame.reset_index(inplace = True)
finalDataFrame.drop('index', axis = 1, inplace = True)

finalDataFrame

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,37.15,2.28,N/A
1,EBAY,42.65,2.39,N/A
2,AMZN,110.00,2.62,N/A
3,MRNA,132.94,3.79,N/A
4,F,11.57,3.93,N/A
5,SYF,30.61,4.11,N/A
6,PARA,25.58,4.11,N/A
7,COF,109.61,4.2,N/A
8,NUE,115.10,4.3,N/A
9,DHI,62.00,4.48,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [47]:
def portfolioInput():
    global portfolioSize
    portfolioSize = input("Enter the value of your portfolio:")

    try:
        portfolioSize = float(portfolioSize)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolioSize = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [48]:
portfolioInput()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [49]:
positionSize = portfolioSize / len(finalDataFrame.index)
print(positionSize)
for i in range(0, len(finalDataFrame.index)):
    finalDataFrame.loc[i, 'Number of Shares to Buy'] = math.floor(positionSize / finalDataFrame.loc[i, 'Price'])

finalDataFrame

200000.0


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,37.15,2.28,5383
1,EBAY,42.65,2.39,4689
2,AMZN,110.00,2.62,1818
3,MRNA,132.94,3.79,1504
4,F,11.57,3.93,17286
5,SYF,30.61,4.11,6533
6,PARA,25.58,4.11,7818
7,COF,109.61,4.2,1824
8,NUE,115.10,4.3,1737
9,DHI,62.00,4.48,3225


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [76]:
symbol = 'AON'

batchApiUrlCall = baseUrl + f'/stock/market/batch?symbols={symbol}&types=price,quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batchApiUrlCall).json()
# print(data)

# Price-to-earnings ratio
peRatio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pbRatio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
psRatio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterpriseValue = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
evToEbitda = enterpriseValue / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
grossProfit = data[symbol]['advanced-stats']['grossProfit']
evToGrossProfit = enterpriseValue / grossProfit

print(evToEbitda)
print(evToGrossProfit)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'NoneType'

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [127]:
rvColumns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-earnings ratio',
    'Price-to-earnings ratio Percentiles',
    'Price-to-book ratio',
    'Price-to-book ratio Percentiles',
    'Price-to-sales ratio',
    'Price-to-sales ratio Percentiles',
    'EV-to-EBITDA ratio',
    'EV-to-EBITDA ratio Percentiles',
    'EV-to-gross-profit ratio',
    'EV-to-gross-profit ratio Percentiles',
    'RV Score',
]

rvDataFrame = pd.DataFrame(columns = rvColumns)

for symbolString in symbolStrings:
    batchApiUrlCall = baseUrl + f'/stock/market/batch?symbols={symbolString}&types=price,quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchApiUrlCall).json()
    for symbol in symbolString.split(','):
        enterpriseValue = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        grossProfit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            evToEbitda = enterpriseValue / ebitda
        except TypeError:
            evToEbitda = np.NaN
        try:
            evToGrossProfit = enterpriseValue / grossProfit
        except TypeError:
            evToGrossProfit = np.NaN
        rvDataFrame = rvDataFrame.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    evToEbitda,
                    'N/A',
                    evToGrossProfit,
                    'N/A',
                    'N/A',
                ],
                index = rvColumns
            ),
            ignore_index = True
        )


rvDataFrame

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,Price-to-earnings ratio Percentiles,Price-to-book ratio,Price-to-book ratio Percentiles,Price-to-sales ratio,Price-to-sales ratio Percentiles,EV-to-EBITDA ratio,EV-to-EBITDA ratio Percentiles,EV-to-gross-profit ratio,EV-to-gross-profit ratio Percentiles,RV Score
0,A,114.37,N/A,28.27,N/A,6.74,N/A,5.34,N/A,19.633617,N/A,9.966529,N/A,N/A
1,AAL,13.51,N/A,-3.54,N/A,-0.9589,N/A,0.2475,N/A,-22.101532,N/A,0.992749,N/A,N/A
2,AAP,168.13,N/A,19.34,N/A,3.6,N/A,0.9426,N/A,10.780869,N/A,2.311521,N/A,N/A
3,AAPL,134.27,N/A,21.7,N/A,31.83,N/A,5.59,N/A,17.156274,N/A,12.985425,N/A,N/A
4,ABBV,141.39,N/A,20.5,N/A,15.24,N/A,4.32,N/A,10.530608,N/A,7.768249,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,110.40,N/A,20.06,N/A,-3.69,N/A,4.9,N/A,19.131043,N/A,8.739855,N/A,N/A
500,ZBH,103.73,N/A,102.61,N/A,1.87,N/A,2.88,N/A,17.846683,N/A,5.139405,N/A,N/A
501,ZBRA,294.37,N/A,19.03,N/A,5.3,N/A,2.71,N/A,15.665946,N/A,6.463573,N/A,N/A
502,ZION,54.78,N/A,5.6,N/A,1.4,N/A,2.31,N/A,4.324871,N/A,2.166243,N/A,N/A


In [128]:
rvDataFrame

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,Price-to-earnings ratio Percentiles,Price-to-book ratio,Price-to-book ratio Percentiles,Price-to-sales ratio,Price-to-sales ratio Percentiles,EV-to-EBITDA ratio,EV-to-EBITDA ratio Percentiles,EV-to-gross-profit ratio,EV-to-gross-profit ratio Percentiles,RV Score
0,A,114.37,N/A,28.27,N/A,6.74,N/A,5.34,N/A,19.633617,N/A,9.966529,N/A,N/A
1,AAL,13.51,N/A,-3.54,N/A,-0.9589,N/A,0.2475,N/A,-22.101532,N/A,0.992749,N/A,N/A
2,AAP,168.13,N/A,19.34,N/A,3.6,N/A,0.9426,N/A,10.780869,N/A,2.311521,N/A,N/A
3,AAPL,134.27,N/A,21.7,N/A,31.83,N/A,5.59,N/A,17.156274,N/A,12.985425,N/A,N/A
4,ABBV,141.39,N/A,20.5,N/A,15.24,N/A,4.32,N/A,10.530608,N/A,7.768249,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,110.40,N/A,20.06,N/A,-3.69,N/A,4.9,N/A,19.131043,N/A,8.739855,N/A,N/A
500,ZBH,103.73,N/A,102.61,N/A,1.87,N/A,2.88,N/A,17.846683,N/A,5.139405,N/A,N/A
501,ZBRA,294.37,N/A,19.03,N/A,5.3,N/A,2.71,N/A,15.665946,N/A,6.463573,N/A,N/A
502,ZION,54.78,N/A,5.6,N/A,1.4,N/A,2.31,N/A,4.324871,N/A,2.166243,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [129]:
rvDataFrame[rvDataFrame.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,Price-to-earnings ratio Percentiles,Price-to-book ratio,Price-to-book ratio Percentiles,Price-to-sales ratio,Price-to-sales ratio Percentiles,EV-to-EBITDA ratio,EV-to-EBITDA ratio Percentiles,EV-to-gross-profit ratio,EV-to-gross-profit ratio Percentiles,RV Score
38,AON,253.18,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
55,BALL,66.93,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
70,BRK.B,272.27,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.73,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
193,FRT,97.16,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2225.06,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
342,NWS,15.98,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
443,TROW,106.96,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
454,UA,8.75,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [130]:
for column in ['Price-to-earnings ratio', 'Price-to-book ratio', 'Price-to-sales ratio', 'EV-to-EBITDA ratio', 'EV-to-gross-profit ratio',
]:
    rvDataFrame[column].fillna(
        rvDataFrame[column].mean(),
        inplace = True
    )

rvDataFrame

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,Price-to-earnings ratio Percentiles,Price-to-book ratio,Price-to-book ratio Percentiles,Price-to-sales ratio,Price-to-sales ratio Percentiles,EV-to-EBITDA ratio,EV-to-EBITDA ratio Percentiles,EV-to-gross-profit ratio,EV-to-gross-profit ratio Percentiles,RV Score
0,A,114.37,N/A,28.27,N/A,6.7400,N/A,5.3400,N/A,19.633617,N/A,9.966529,N/A,N/A
1,AAL,13.51,N/A,-3.54,N/A,-0.9589,N/A,0.2475,N/A,-22.101532,N/A,0.992749,N/A,N/A
2,AAP,168.13,N/A,19.34,N/A,3.6000,N/A,0.9426,N/A,10.780869,N/A,2.311521,N/A,N/A
3,AAPL,134.27,N/A,21.70,N/A,31.8300,N/A,5.5900,N/A,17.156274,N/A,12.985425,N/A,N/A
4,ABBV,141.39,N/A,20.50,N/A,15.2400,N/A,4.3200,N/A,10.530608,N/A,7.768249,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,110.40,N/A,20.06,N/A,-3.6900,N/A,4.9000,N/A,19.131043,N/A,8.739855,N/A,N/A
500,ZBH,103.73,N/A,102.61,N/A,1.8700,N/A,2.8800,N/A,17.846683,N/A,5.139405,N/A,N/A
501,ZBRA,294.37,N/A,19.03,N/A,5.3000,N/A,2.7100,N/A,15.665946,N/A,6.463573,N/A,N/A
502,ZION,54.78,N/A,5.60,N/A,1.4000,N/A,2.3100,N/A,4.324871,N/A,2.166243,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [131]:
rvDataFrame[rvDataFrame.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,Price-to-earnings ratio Percentiles,Price-to-book ratio,Price-to-book ratio Percentiles,Price-to-sales ratio,Price-to-sales ratio Percentiles,EV-to-EBITDA ratio,EV-to-EBITDA ratio Percentiles,EV-to-gross-profit ratio,EV-to-gross-profit ratio Percentiles,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [132]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-earnings ratio': 'Price-to-earnings ratio Percentiles',
    'Price-to-book ratio': 'Price-to-book ratio Percentiles',
    'Price-to-sales ratio': 'Price-to-sales ratio Percentiles',
    'EV-to-EBITDA ratio': 'EV-to-EBITDA ratio Percentiles',
    'EV-to-gross-profit ratio': 'EV-to-gross-profit ratio Percentiles',
}

for metric in metrics.keys():
    for row in rvDataFrame.index:
        rvDataFrame.loc[row, metrics[metric]] = score(
            rvDataFrame[metric],
            rvDataFrame.loc[row, metric]
        ) / 100
        
rvDataFrame

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,Price-to-earnings ratio Percentiles,Price-to-book ratio,Price-to-book ratio Percentiles,Price-to-sales ratio,Price-to-sales ratio Percentiles,EV-to-EBITDA ratio,EV-to-EBITDA ratio Percentiles,EV-to-gross-profit ratio,EV-to-gross-profit ratio Percentiles,RV Score
0,A,114.37,N/A,28.27,0.72619,6.7400,0.803571,5.3400,0.80754,19.633617,0.767857,9.966529,0.769841,N/A
1,AAL,13.51,N/A,-3.54,0.027778,-0.9589,0.063492,0.2475,0.009921,-22.101532,0.007937,0.992749,0.037698,N/A
2,AAP,168.13,N/A,19.34,0.486111,3.6000,0.592262,0.9426,0.146825,10.780869,0.363095,2.311521,0.142857,N/A
3,AAPL,134.27,N/A,21.70,0.549603,31.8300,0.970238,5.5900,0.819444,17.156274,0.678571,12.985425,0.869048,N/A
4,ABBV,141.39,N/A,20.50,0.509921,15.2400,0.928571,4.3200,0.72619,10.530608,0.347222,7.768249,0.632937,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,110.40,N/A,20.06,0.501984,-3.6900,0.05754,4.9000,0.785714,19.131043,0.748016,8.739855,0.694444,N/A
500,ZBH,103.73,N/A,102.61,0.984127,1.8700,0.287698,2.8800,0.531746,17.846683,0.704365,5.139405,0.386905,N/A
501,ZBRA,294.37,N/A,19.03,0.47619,5.3000,0.724206,2.7100,0.496032,15.665946,0.623016,6.463573,0.505952,N/A
502,ZION,54.78,N/A,5.60,0.079365,1.4000,0.191468,2.3100,0.425595,4.324871,0.065476,2.166243,0.134921,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [133]:
from statistics import mean

for row in rvDataFrame.index:
    percentiles = []
    for metric in metrics:
        percentiles.append(rvDataFrame.loc[row, metrics[metric]])
    rvDataFrame.loc[row, 'RV Score'] = mean(percentiles)
    
rvDataFrame

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,Price-to-earnings ratio Percentiles,Price-to-book ratio,Price-to-book ratio Percentiles,Price-to-sales ratio,Price-to-sales ratio Percentiles,EV-to-EBITDA ratio,EV-to-EBITDA ratio Percentiles,EV-to-gross-profit ratio,EV-to-gross-profit ratio Percentiles,RV Score
0,A,114.37,N/A,28.27,0.72619,6.7400,0.803571,5.3400,0.80754,19.633617,0.767857,9.966529,0.769841,0.775
1,AAL,13.51,N/A,-3.54,0.027778,-0.9589,0.063492,0.2475,0.009921,-22.101532,0.007937,0.992749,0.037698,0.029365
2,AAP,168.13,N/A,19.34,0.486111,3.6000,0.592262,0.9426,0.146825,10.780869,0.363095,2.311521,0.142857,0.34623
3,AAPL,134.27,N/A,21.70,0.549603,31.8300,0.970238,5.5900,0.819444,17.156274,0.678571,12.985425,0.869048,0.777381
4,ABBV,141.39,N/A,20.50,0.509921,15.2400,0.928571,4.3200,0.72619,10.530608,0.347222,7.768249,0.632937,0.628968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,110.40,N/A,20.06,0.501984,-3.6900,0.05754,4.9000,0.785714,19.131043,0.748016,8.739855,0.694444,0.55754
500,ZBH,103.73,N/A,102.61,0.984127,1.8700,0.287698,2.8800,0.531746,17.846683,0.704365,5.139405,0.386905,0.578968
501,ZBRA,294.37,N/A,19.03,0.47619,5.3000,0.724206,2.7100,0.496032,15.665946,0.623016,6.463573,0.505952,0.565079
502,ZION,54.78,N/A,5.60,0.079365,1.4000,0.191468,2.3100,0.425595,4.324871,0.065476,2.166243,0.134921,0.179365


## Selecting the 50 Best Value Stocks

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [134]:
rvDataFrame.sort_values(
    'RV Score',
    ascending = True,
    inplace = True
)
rvDataFrame = rvDataFrame[:50]
rvDataFrame.reset_index(inplace = True, drop = True)
rvDataFrame

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,Price-to-earnings ratio Percentiles,Price-to-book ratio,Price-to-book ratio Percentiles,Price-to-sales ratio,Price-to-sales ratio Percentiles,EV-to-EBITDA ratio,EV-to-EBITDA ratio Percentiles,EV-to-gross-profit ratio,EV-to-gross-profit ratio Percentiles,RV Score
0,AAL,13.51,N/A,-3.54,0.027778,-0.9589,0.063492,0.2475,0.009921,-22.101532,0.007937,0.992749,0.037698,0.029365
1,GM,32.23,N/A,5.56,0.077381,0.7551,0.079365,0.3647,0.027778,1.789262,0.017857,1.497469,0.085317,0.05754
2,AIG,53.60,N/A,4.60,0.056548,0.7429,0.077381,0.7744,0.113095,2.806842,0.029762,0.755630,0.015873,0.058532
3,LNC,47.85,N/A,7.13,0.107143,0.5608,0.069444,0.4237,0.035714,5.122670,0.10119,0.437316,0.001984,0.063095
4,PVH,63.46,N/A,4.60,0.056548,0.8108,0.083333,0.4562,0.043651,4.619303,0.083333,1.087966,0.051587,0.06369
5,F,11.38,N/A,4.10,0.046627,1.0067,0.117063,0.3295,0.017857,3.523599,0.041667,1.813050,0.105159,0.065675
6,PRU,92.03,N/A,7.35,0.113095,0.8155,0.087302,0.5276,0.055556,4.692134,0.085317,0.522367,0.005952,0.069444
7,SYF,30.50,N/A,4.10,0.046627,1.2300,0.150794,0.9003,0.134921,2.677655,0.025794,0.930497,0.029762,0.077579
8,MET,62.51,N/A,7.93,0.128968,0.9396,0.111111,0.7142,0.095238,4.409925,0.073413,0.689974,0.011905,0.084127
9,PARA,25.26,N/A,4.06,0.043651,0.7144,0.075397,0.5899,0.06746,7.411126,0.212302,0.932471,0.031746,0.086111


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [135]:
portfolioInput()

Enter the value of your portfolio:987654321


In [136]:
positionSize = portfolioSize / len(rvDataFrame.index)
print(f'positionSize: {positionSize}')

for i in range(0, len(rvDataFrame.index)):
    rvDataFrame.loc[i, 'Number of Shares to Buy'] = math.floor(positionSize / rvDataFrame.loc[i, 'Price'])

rvDataFrame

positionSize: 19753086.42


/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,Price-to-earnings ratio Percentiles,Price-to-book ratio,Price-to-book ratio Percentiles,Price-to-sales ratio,Price-to-sales ratio Percentiles,EV-to-EBITDA ratio,EV-to-EBITDA ratio Percentiles,EV-to-gross-profit ratio,EV-to-gross-profit ratio Percentiles,RV Score
0,AAL,13.51,1462108,-3.54,0.027778,-0.9589,0.063492,0.2475,0.009921,-22.101532,0.007937,0.992749,0.037698,0.029365
1,GM,32.23,612878,5.56,0.077381,0.7551,0.079365,0.3647,0.027778,1.789262,0.017857,1.497469,0.085317,0.05754
2,AIG,53.60,368527,4.60,0.056548,0.7429,0.077381,0.7744,0.113095,2.806842,0.029762,0.755630,0.015873,0.058532
3,LNC,47.85,412812,7.13,0.107143,0.5608,0.069444,0.4237,0.035714,5.122670,0.10119,0.437316,0.001984,0.063095
4,PVH,63.46,311268,4.60,0.056548,0.8108,0.083333,0.4562,0.043651,4.619303,0.083333,1.087966,0.051587,0.06369
5,F,11.38,1735772,4.10,0.046627,1.0067,0.117063,0.3295,0.017857,3.523599,0.041667,1.813050,0.105159,0.065675
6,PRU,92.03,214637,7.35,0.113095,0.8155,0.087302,0.5276,0.055556,4.692134,0.085317,0.522367,0.005952,0.069444
7,SYF,30.50,647642,4.10,0.046627,1.2300,0.150794,0.9003,0.134921,2.677655,0.025794,0.930497,0.029762,0.077579
8,MET,62.51,315998,7.93,0.128968,0.9396,0.111111,0.7142,0.095238,4.409925,0.073413,0.689974,0.011905,0.084127
9,PARA,25.26,781990,4.06,0.043651,0.7144,0.075397,0.5899,0.06746,7.411126,0.212302,0.932471,0.031746,0.086111


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [166]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rvDataFrame.to_excel(
    writer,
    sheet_name = 'Value Strategy',
    index = False
)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [167]:
backgroundColor = '#0a0a23'
fontColor = '#ffffff'

stringTemplate = writer.book.add_format(
        {
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

dollarTemplate = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

integerTemplate = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )
floatTemplate = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )
percentTemplate = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

In [168]:
rvColumnFormats = {
    'A': ['Ticker', stringTemplate],
    'B': ['Price', dollarTemplate],
    'C': ['Number of Shares to Buy', integerTemplate],
    'D': ['Price-to-earnings ratio', floatTemplate],
    'E': ['Price-to-earnings ratio Percentiles', percentTemplate],
    'F': ['Price-to-book ratio', floatTemplate],
    'G': ['Price-to-book ratio Percentiles', percentTemplate],
    'H': ['Price-to-sales ratio', floatTemplate],
    'I': ['Price-to-sales ratio Percentiles', percentTemplate],
    'J': ['EV-to-EBITDA ratio', floatTemplate],
    'K': ['EV-to-EBITDA ratio Percentiles', percentTemplate],
    'L': ['EV-to-gross-profit ratio', floatTemplate],
    'M': ['EV-to-gross-profit ratio Percentiles', percentTemplate],
    'N': ['RV Score', percentTemplate],
}

for column in rvColumnFormats.keys():
    writer.sheets['Value Strategy'].set_column(
        f'{column}:{column}',
        max(len(rvColumnFormats[column][0]), 10),
        rvColumnFormats[column][1]
    )
    writer.sheets['Value Strategy'].write(
        f'{column}1',
        rvColumnFormats[column][0],
        rvColumnFormats[column][1]
    )

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [169]:
writer.save()